# Chapter 4: Counting Elements

Store the data in a slightly different way, by making an array of counters. Each number may be counted in the array by using an index that corresponds to the value of the given number. Notice that we do not place elements directly into a cell; rather, we simply count their occurrences. It is important that the array in which we count elements is sufficiently large. If we know that all the elements are in the set {0,1,...,m}, then the array used for counting should be of size m + 1.

In [6]:
import time

In [1]:
# Counting elements — O(n + m).
def counting(A,m):
    n = len(A)
    count = [0]*(m+1)
    for k in xrange(n):
        print 'count',count
        count[A[k]] += 1
    return count

In [2]:
counting([0,0,4,2,4,5],5)

count [0, 0, 0, 0, 0, 0]
count [1, 0, 0, 0, 0, 0]
count [2, 0, 0, 0, 0, 0]
count [2, 0, 0, 0, 1, 0]
count [2, 0, 1, 0, 1, 0]
count [2, 0, 1, 0, 2, 0]


[2, 0, 1, 0, 2, 1]

The limitation here may be available memory. Usually, we are not able to create arrays of 10^9 integers, because this would require more than 1GB of available memory.

__Problem__ 
You are given an integer m (1 <= m <= 1000000) and two non-empty, zero-indexed arrays A and B of n integers, a[0] ,a[1] ,...,a[n−1] and b[0] ,b[1] ,...,b[n−1] respectively (0 <= a[i] ,b[i] <= m). The goal is to check whether there is a swap operation which can be performed on these arrays in such a way that the sum of elements in array A equals the sum of elements in array B after the swap. By swap operation we mean picking one element from array A and one element from array B and exchanging them.

__Solution__ 
O(n^2): The simplest method is to swap every pair of elements and calculate the totals. Using that approach gives us O(n^2) time complexity. A better approach is to calculate the sums of elements at the beginning, and check only how the totals change during the swap operation.

In [40]:
# Swap the elements — O(n^2)
def slow_solution(A, B, m):
    n = len(A)
    sum_a = sum(A)
    sum_b = sum(B)
    for i in xrange(n):
        for j in xrange(n):
            change = B[j] - A[i]
            sum_a += change
            sum_b -= change
            if sum_a == sum_b:
                return True
            sum_a -= change
            sum_b += change
    return False

In [41]:
start_time = time.time()

print slow_solution([3,5,7,4,7,6,4,4],[4,6,1,8,5,4,9,3],7)

end_time = time.time()
print("Elapsed time was %g seconds" % (end_time - start_time))

True
Elapsed time was 0.000999928 seconds


__Better Solution__ O(n + m): Best approach is to count the elements of array A and calculate the difference d between the sums of the elements of array A and B. For every element of array B, we assume that we will swap it with some element from array A. The difference d tells us the value from array A that we are interested in swapping, because only one value will cause the two totals to be equal. The occurrence of this value can be found in constant time from the array used for counting.

In [13]:
# Swap the elements — O(n + m)
def fast_solution(A, B, m):
    n = len(A)
    sum_a = sum(A)
    sum_b = sum(B)
    d = sum_b - sum_a
    print 'sum_a',sum_a,'sum_b',sum_b, 'd',d
    if d % 2 == 1:
        return False
    d //= 2
    print 'd',d   
    count = counting(A, m) # count elements of A
    for i in xrange(n):
        print 'i',i
        print 'B[i]',B[i],'count[B[i]-d]',count[B[i] - d]
        if 0 <= B[i] - d and B[i] - d <= m and count[B[i] - d] > 0:
            return True
    return False

In [18]:
print fast_solution([1,2,2,1,3],[2,6,5,8,6],8)

sum_a 9 sum_b 27 d 18
d 9
count [0, 0, 0, 0, 0, 0, 0, 0, 0]
count [0, 1, 0, 0, 0, 0, 0, 0, 0]
count [0, 1, 1, 0, 0, 0, 0, 0, 0]
count [0, 1, 2, 0, 0, 0, 0, 0, 0]
count [0, 2, 2, 0, 0, 0, 0, 0, 0]
i 0
B[i] 2 count[B[i]-d] 2
i 1
B[i] 6 count[B[i]-d] 0
i 2
B[i] 5 count[B[i]-d] 0
i 3
B[i] 8 count[B[i]-d] 0
i 4
B[i] 6 count[B[i]-d] 0
False


In [19]:
start_time = time.time()

print fast_solution([3,5,7,4,7,6,4,4],[4,6,1,8,5,4,9,3],7)

end_time = time.time()
print("Elapsed time was %g seconds" % (end_time - start_time))

sum_a 40 sum_b 40 d 0
d 0
count [0, 0, 0, 0, 0, 0, 0, 0]
count [0, 0, 0, 1, 0, 0, 0, 0]
count [0, 0, 0, 1, 0, 1, 0, 0]
count [0, 0, 0, 1, 0, 1, 0, 1]
count [0, 0, 0, 1, 1, 1, 0, 1]
count [0, 0, 0, 1, 1, 1, 0, 2]
count [0, 0, 0, 1, 1, 1, 1, 2]
count [0, 0, 0, 1, 2, 1, 1, 2]
i 0
B[i] 4 count[B[i]-d] 3
True
Elapsed time was 0.00200009 seconds


# Chapter 5: prefix sums

There is a simple yet powerful technique that allows for the fast calculation of sums of elements in given slice (contiguous segments of array). Its main idea uses prefix sums which are defined as the consecutive totals of the first 0,1,2,...,n elements of an array. We can easily calculate the prefix sums in O(n) time complexity. Notice that the total p[k] equals p[k−1] + a[k−1],  so each consecutive value can be calculated in a constant time.

Similarly, we can calculate suffix sums, which are the totals of the k last values. Using prefix (or suffix) sums allows us to calculate the total of any slice of the array very quickly. For example, assume that you are asked about the totals of m slices [x..y] such that 0 ‹ x ‹ y < n, where the total is the sum a[x] + a[x+1] + ... + a[y−1] + a[y]. The simplest approach is to iterate through the whole array for each result separately; however, that requires O(n·m) time. The better approach is to use prefix sums. If we calculate the prefix sums then we can answer each question directly in constant time.

In [47]:
# Counting prefix sums — O(n).prefix_sums
def prefix_sums(A):
    n = len(A)
    P = [0]*(n + 1)
    for k in xrange(1, n + 1):
        P[k] = P[k - 1] + A[k - 1]
        print 'P[k]',P[k],'P[k - 1]',P[k - 1],'A[k - 1]',A[k - 1]
    return P

# Total of one slice — O(1).
def count_total(P, x, y):
    return P[y + 1] - P[x]

In [48]:
prefix_sums([2,3,7,5,1,3,9])

P[k] 2 P[k - 1] 0 A[k - 1] 2
P[k] 5 P[k - 1] 2 A[k - 1] 3
P[k] 12 P[k - 1] 5 A[k - 1] 7
P[k] 17 P[k - 1] 12 A[k - 1] 5
P[k] 18 P[k - 1] 17 A[k - 1] 1
P[k] 21 P[k - 1] 18 A[k - 1] 3
P[k] 30 P[k - 1] 21 A[k - 1] 9


[0, 2, 5, 12, 17, 18, 21, 30]

A mushroom picker is at spot number k on the road and should perform m moves. In one move she moves to an adjacent spot. She collects all the mushrooms growing on spots she visits. The goal is to calculate the maximum number of mushrooms that the mushroom picker can collect in m moves.

The mushroom picker starts at spot k = 4 and should perform m = 6 moves. She might move to spots 3,2,3,4,5,6 and thereby collect 1+5+7+3+9 = 25 mushrooms. This is the maximal number of mushrooms she can collect.

Solution O(m^2): Note that the best strategy is to move in one direction optionally followed by some moves in the opposite direction. In other words, the mushroom picker should not change direction more than once. With this observation we can find the simplest solution. Make the first p = 0,1,2,...,m moves in one direction, then the next m − p moves in the opposite direction. This is just a simple simulation of the moves of the mushroom picker which requires O(m^2) time.

Solution O(n+m): A better approach is to use prefix sums. If we make p moves in one direction, we can calculate the maximal opposite location of the mushroom picker. The mushroom picker collects all mushrooms between these extremes. We can calculate the total number of collected mushrooms in constant time by using prefix sums.

In [45]:
# 5.3: Mushroom picker — O(n + m)
def mushrooms(A, k, m):
    n = len(A)
    result = 0
    pref = prefix_sums(A)
    for p in xrange(min(m, k) + 1):
        left_pos = k - p
        right_pos = min(n - 1, max(k, k + m - 2 * p))
        result = max(result, count_total(pref, left_pos, right_pos))
    for p in xrange(min(m + 1, n - k)):
        right_pos = k + p
        left_pos = max(0, min(k, k - (m - 2 * p)))
        result = max(result, count_total(pref, left_pos, right_pos))
    return result

In [35]:
mushrooms([2,3,7,5,1,3,9],4,6)

25

# Chapter 6: sorting

__Selection sort__

The idea: Find the minimal element and swap it with the first element of an array. Next, just sort the rest of the array, without the first element, in the same way. Notice that after k iterations the first k elements will be sorted in the right order (this property is called the invariant).

In [49]:
# Selection sort — O(n^2).
def selectionSort(A):
    n = len(A)
    for k in xrange(n):
        minimal = k
        for j in xrange(k + 1, n):
            if (A[minimal] > A[j]):
                minimal = j
        A[k], A[minimal] = A[minimal], A[k]
    return A

In [53]:
alist = [54,26,93,17,77,31,44,55,20]
selectionSort(alist)
print(alist)

[17, 20, 26, 31, 44, 54, 55, 77, 93]


__Counting sort__

The idea: First, count the elements in the array of counters (see chapter 2). Next, just iterate through the array of counters in increasing order. Notice that we have to know the range of the sorted values. If all the elements are in the set {0,1,...,k}, then the array used for counting should be of size k+1. The limitation here may be available memory.

In [50]:
#  Counting sort — O(n + k)
def countingSort(A, k):
    n = len(A)
    count = [0] * (k + 1)
    for i in xrange(n):
        count[A[i]] += 1
    p = 0
    for i in xrange(k + 1):
        for j in xrange(count[i]):
            A[p] = i;
            p += 1;
    return A

In [57]:
alist = [54,26,93,17,77,31,44,55,20]
countingSort(alist,9)
print(alist)

IndexError: list index out of range

__Merge sort__

The idea: Divide the unsorted array into two halves, sort each half separately and then just merge them.

After the split, each part is halved again. We repeat this algorithm until we end up with individual elements, which are sorted by definition. The merging of two sorted arrays consisting of n elements takes O(n) time; just repeatedly choose the lower of the first elements of the two merged parts.

The number of divisions will be O(n), because the length of the array is halved on each iteration. In this way, we get consecutive levels with 1,2,4,8,... slices. For each level, the merging of the all consecutive pairs of slices requires O(n) time. The number of levels is O(logn), so the total time complexity is O(nlogn)

In [51]:
def mergeSort(alist):
    print("Splitting ",alist)
    if len(alist)>1:
        mid = len(alist)//2
        lefthalf = alist[:mid]
        righthalf = alist[mid:]

        mergeSort(lefthalf)
        mergeSort(righthalf)

        i=0
        j=0
        k=0
        while i < len(lefthalf) and j < len(righthalf):
            if lefthalf[i] < righthalf[j]:
                alist[k]=lefthalf[i]
                i=i+1
            else:
                alist[k]=righthalf[j]
                j=j+1
            k=k+1

        while i < len(lefthalf):
            alist[k]=lefthalf[i]
            i=i+1
            k=k+1

        while j < len(righthalf):
            alist[k]=righthalf[j]
            j=j+1
            k=k+1
    print("Merging ",alist)

In [52]:
alist = [54,26,93,17,77,31,44,55,20]
mergeSort(alist)
print(alist)

('Splitting ', [54, 26, 93, 17, 77, 31, 44, 55, 20])
('Splitting ', [54, 26, 93, 17])
('Splitting ', [54, 26])
('Splitting ', [54])
('Merging ', [54])
('Splitting ', [26])
('Merging ', [26])
('Merging ', [26, 54])
('Splitting ', [93, 17])
('Splitting ', [93])
('Merging ', [93])
('Splitting ', [17])
('Merging ', [17])
('Merging ', [17, 93])
('Merging ', [17, 26, 54, 93])
('Splitting ', [77, 31, 44, 55, 20])
('Splitting ', [77, 31])
('Splitting ', [77])
('Merging ', [77])
('Splitting ', [31])
('Merging ', [31])
('Merging ', [31, 77])
('Splitting ', [44, 55, 20])
('Splitting ', [44])
('Merging ', [44])
('Splitting ', [55, 20])
('Splitting ', [55])
('Merging ', [55])
('Splitting ', [20])
('Merging ', [20])
('Merging ', [20, 55])
('Merging ', [20, 44, 55])
('Merging ', [20, 31, 44, 55, 77])
('Merging ', [17, 20, 26, 31, 44, 54, 55, 77, 93])
[17, 20, 26, 31, 44, 54, 55, 77, 93]


# Chapter 7: stacks

__Stack__

The stack is a basic data structure in which the insertion of new elements takes place at the top and deletion of elements also takes place from the top. The idea of the stack can be illustrated by plates stacked on top of one another. Each new plate is placed on top of the stack of plates, and plates can only be taken off the top of the stack.

The stack can be represented by an array in which we should also remember the size of the stack and we must be sure to declare sufficient space for the array (in the following implementation we can store N elements).

In [59]:
# push / pop operations — O(1).
N = 100
stack = [0]*N
size = 0
def push(x):
    global size
    stack[size] = x
    size += 1
def pop():
    global size
    size -= 1
    return stack[size]

__Queue__

The queue is a basic data structure in which new elements are inserted at the back but old elements are removed from the front. The idea of the queue can be illustrated by a line of customers in a grocery store. New people join the back of the queue and the next person to be served is the first one in the line.

In [60]:
#  Push / pop / size / empty operations — O(1).
N = 100
queue = [0] * N
head, tail = 0, 0
def push(x):
    global tail
    tail = (tail + 1) % N
    queue[tail] = x
def pop():
    global head
    head = (head + 1) % N
    return queue[head]
def size():
    return (tail - head + N) % N
def empty():
    return head == tail

Problem: You are given a zero-indexed array A consisting of n integers: a[0] ,a[1] ,...,a[n−1].
Array A represents a scenario in a grocery store, and contains only 0s and/or 1s:

* 0 represents the action of a new person joining the line in the grocery store,
* 1 represents the action of the person at the front of the queue being served and leaving the line.

The goal is to count the minimum number of people who should have been in the line before the above scenario, so that the scenario is possible (it is not possible to serve a person if the line is empty).

Solution: We should remember the size of the queue and carry out a simulation of people arriving at and leaving the grocery store. If the size of the queue becomes a negative number then that sets the lower limit for the number of people who had to stand in the line previously. We should find the smallest negative number to determine the size of the queue during the whole simulation.

In [62]:
# Golden solution — O(n)
def groceryStore(A):
    n = len(A)
    size, result = 0, 0
    for i in xrange(n):
        if (A[i] == 0):
            size += 1
        else:
            size -= 1
            result = max(result, -size);
    return result

In [63]:
groceryStore([1,0,1,0,1,1,0,0,1,1,1,1])

4

# Chapter 8: leader

Let us consider a sequence a[0] ,a[1] ,...,a[n-1]. The leader of this sequence is the element whose value occurs more than
n/2 times.

|a[0]| a[1]| a[2]| a[3]| a[4]| a[5]| a[6]|
|---|---|---|---|---|
|**6**   | 8   | 4    |**6**    |8   | **6** | **6**|
|0   | 1   | 2    |3   | 4   | 5 |   6|

In the picture the leader is highlighted in gray. Notice that the sequence can have at most one leader. If there were two leaders then their total occurrences would be more than 2 · n/2 = n, but we only have n elements.
The leader may be found in many ways. We describe some methods here, starting with trivial, slow ideas and ending with very creative, fast algorithms. The task is to find the value of the leader of the sequence a[0] ,a[1] ,...,a[n-1] , such that 0 <= a <= 10^9. If there is no leader, the result should be -1

In [66]:
# Leader — O(n^2).
def slowLeader(A):
    n = len(A)
    leader = -1
    for k in xrange(n):
        candidate = A[k]
        count = 0
        for i in xrange(n):
            if (A[i] == candidate):
                count += 1
        if (count > n // 2):
            leader = candidate
    return leader

If the sequence is presented in non-decreasing order, then identical values are adjacent to each other.

|a[0]| a[1]| a[2]| a[3]| a[4]| a[5]| a[6]|
|---|---|---|---|---|
|4   | **6**   | **6**    |**6**    |**6**   | 8 | 8|
|0   | 1   | 2    |*3*   | 4   | 5 |   6|

Having sorted the sequence, we can easily count slices of the same values and find the leader in a smarter way. Notice that if the leader occurs somewhere in our sequence, then it must occur at index n/2 (the central element). This is because, given that the leader occurs in more than half the total values in the sequence, there are more leader values than will fit on either side of the central element in the sequence.

In [68]:
# Leader — O(nlogn).
def fastLeader(A):
    n = len(A)
    leader = -1
    A.sort()
    candidate = A[n // 2]
    count = 0
    for i in xrange(n):
        if (A[i] == candidate):
            count += 1
    if (count > n // 2):
        leader = candidate
    return leader

At the beginning we notice that if the sequence contains a leader, then after the removal of different elements the leader will not have changed. After removing all pairs of different elements, we end up with a sequence containing all the same values. This value is not necessarily the leader; it is only a candidate for the leader. Finally, we should iterate through all the elements and count the occurrences of the candidate; if it is greater than n/2 then we have found the leader; otherwise the sequence does not contain a leader. The time complexity of this algorithm is O(n) because every element is considered only once. The final counting of occurrences of the candidate value also works in O(n) time

In [69]:
# Leader — O(n).
def goldenLeader(A):
    n = len(A)
    size = 0
    for k in xrange(n):
        if (size == 0):
            size += 1
            value = A[k]
        else:
            if (value != A[k]):
                size -= 1
            else:
                size += 1
    candidate = -1
    if (size > 0):
        candidate = value
    leader = -1
    count = 0
    for k in xrange(n):
        if (A[k] == candidate):
            count += 1
    if (count > n // 2):
        leader = candidate
    return leader

# Chapter 9: Maximum slice problem

Let’s define a problem relating to maximum slices. You are given a sequence of n integers a[0] ,a[1] ,...,a[n-1] and the task is to find the slice with the largest sum. More precisely, we are looking for two indices p,q such that the total a[p] + a[p+1] + ... + a[q] is maximal. We assume that the slice can be empty and its sum equals 0.

|a[0]| a[1]| a[2]| a[3]| a[4]| a[5]| a[6]|
|---|---|---|---|---|---|---|
|5 |-7 |**3**| **5** |**-2** |**4** |-1 |

In the picture, the slice with the largest sum is highlighted in gray. The sum of this slice equals 10 and there is no slice with a larger sum. Notice that the slice we are looking for may contain negative integers, as shown above

In [71]:
# Maximal slice — O(n^3).
def slow_max_slice(A):
    n = len(A)
    result = 0
    for p in xrange(n):
        for q in xrange(p, n):
            sum = 0
            for i in xrange(p, q + 1):
                sum += A[i]
            result = max(result, sum)
    return result

Analyzing all possible slices requires O(n^2) time complexity, and for each of them we compute the total in O(n) time complexity. It is the most straightforward solution, however it is far from optimal

We can easily improve our last solution. Notice that the prefix sum allows the sum of any slice to be computed in a constant time. With this approach, the time complexity of the whole algorithm reduces to O(n^2). We assume that pref is an array of prefix sums (pref_i = a[0] + a[1] + ... + a[i-1]).

In [73]:
# Maximal slice — O(n^2).
def quadratic_max_slice(A, pref):
    n = len(A) 
    result = 0
    for p in xrange(n):
        for q in xrange(p, n):
            sum = pref[q + 1] - pref[p]
            result = max(result, sum)
    return result

We can also solve this problem without using prefix sums, within the same time complexity. Assume that we know the sum of slice (p,q), so s = a[p] + a[p+1] +...+ a[q]. The sum of the slice with one more element (p,q+1) equals s + a[q+1]. Following this observation, there is no need to compute the sum each time from the beginning; we can use the previously calculated sum.

In [75]:
# Maximal slice — O(n^2).
def quadratic_max_slice(A):
    n = len(A)
    result = 0
    for p in xrange(n):
        sum = 0
        for q in xrange(p, n):
            sum += A[q]
            result = max(result, sum)
    return result

This problem can be solved even faster. For each position, we compute the largest sum that ends in that position. If we assume that the maximum sum of a slice ending in position i equals max_ending, then the maximum slice ending in position i+1 equals max(0,max_ending + a[i+1]).

This time, the fastest algorithm is the one with the simplest implementation, however it is conceptually more difficult. We have used here a very popular and important technique. Based on the solution for shorter sequences we can find the solution for longer sequences.

In [76]:
# Maximal slice — O(n).
def golden_max_slice(A):
    max_ending = max_slice = 0
    for a in A:
        max_ending = max(0, max_ending + a)
        max_slice = max(max_slice, max_ending)
    return max_slice

# Chapter 10: Prime and composite numbers

Let’s count the number of divisors of n. The easiest approach is to iterate through all the numbers from 1 to n and check whether or not each one is a divisor. The time complexity of this solution is O(n).

There is a simple way to improve the above solution. Based on one divisor, we can find the symmetric divisor. More precisely, if number a is a divisor of n, then n/a is also a divisor. One of these two divisors is less than or equal to √n. (If that were not the case, n would be a product of two numbers greater than √ n, which is impossible.)

Thus, iterating through all the numbers from 1 to √n allows us to find all the divisors. If number n is of the form k 2 , then the symmetric divisor of k is also k. This divisor should be counted just once.

In [77]:
# Counting the number of divisors — O(√n).
def divisors(n):
    i = 1
    result = 0
    while (i * i < n):
        if (n % i == 0):
            result += 2
        i += 1
    if (i * i == n):
        result += 1
    return result

The primality test of n can be performed in an analogous way to counting the divisors. If we find a number between 2 and n−1 that divides n then n is a composite number. Otherwise, n is a prime number.

We assume that 1 is neither a prime nor a composite number, so the above algorithm works
only for n › 2.

In [78]:
# Primality test — O( √ n).
def primality(n):
    i = 2
    while (i * i <= n):
        if (n % i == 0):
            return False
        i += 1
    return True

Problem: Consider n coins aligned in a row. Each coin is showing heads at the beginning.

1 2 3 4 5 6 7 8 9 10

Then, n people turn over corresponding coins as follows. Person i reverses coins with numbers that are multiples of i. That is, person i flips coins i, 2·i, 3·i, ... until no more appropriate coins remain. The goal is to count the number of coins showing tails. In the above example, the final configuration is:

**1** 2 3 **4** 5 6 7 8 **9** 10

Solution O(nlogn): We can simulate the results of each person reversing coins.

In [79]:
# Reversing coins — O(nlogn).
def coins(n):
    result = 0
    coin = [0]*(n + 1)
    for i in xrange(1, n + 1):
        k = i
        while (k <= n):
            coin[k] = (coin[k] + 1) % 2
            k += i
        result += coin[i]
    return resul

The number of operation can be estimated by n/1 + n/2 + ... + n/n, what equals n·(1/1 + 1/2 + ... + 1/n). The sums of multiplicative inverses (reciprocals) of the first n numbers are called harmonic numbers, which asymptotically equal O(logn). In summary, the total time complexity is O(nlogn).

Solution O(logn): Notice that each coin will be turned over exactly as many times as the number of its divisors. The coins that are reversed an odd number of times show tails, meaning that it is sufficient to find the coins with an odd number of divisors.
We know that almost every number has a symmetric divisor (apart from divisors of the form √n). Thus, every number of the form k 2 has an odd number of divisors. There are exactly |√n| such numbers between 1 and n. Finding the value of |√n| takes logarithmic time (or constant time if we use operations on floating point numbers).

#Chapter 11 Sieve of Eratosthenes

The Sieve of Eratosthenes is a very simple and popular technique for finding all the prime numbers in the range from 2 to a given number n. The algorithm takes its name from the process of sieving—in a simple way we remove multiples of consecutive numbers.

Initially, we have the set of all the numbers {2,3,...,n}. At each step we choose the smallest number in the set and remove all its multiples. Notice that every composite number has a divisor of at most sqrt(n). In particular, it has a divisor which is a prime number. It is su?cient to remove only multiples of prime numbers not exceeding sqrt(n). In this way, all composite numbers will be removed.

The above algorithm can be slightly improved. Notice that we needn’t cross out multiples of i which are less than i^2. Such multiples are of the form k · i, where k < i. These have already been removed by one of the prime divisors of k. After this improvement, we obtain the following implementation

In [81]:
# O(n*log(log(n)))
def sieve(n):
    sieve = [True] * (n + 1)
    sieve[0] = sieve[1] = False
    i = 2
    while (i * i <= n):
        if (sieve[i]):
            k = i * i
            while (k <= n):
                sieve[k] = False
                k += i
        i += 1
    return sieve

Factorization is the process of decomposition into prime factors. More precisely, for a given number x we want to find primes p[1] ,p[2] ,...,p[k] whose product equals x. Use of the sieve enables fast factorization. Let’s modify the sieve algorithm slightly For every crossed number we will remember the smallest prime that divides this number.

In [83]:
# Preparing the array F for factorization.
def arrayF(n):
    F = [0] * (n + 1)
    i = 2
    while (i * i <= n):
        if (F[i] == 0):
            k = i * i
            while (k <= n):
                if (F[k] == 0):
                    F[k] = i;
                k += i
        i += 1
    return F

# Factorization of x — O(logx).
def factorization(x, F):
    primeFactors = []
    while (F[x] > 0):
        primeFactors += [F[x]]
        x /= F[x]
    primeFactors += [x]
    return primeFactors

With this approach we can factorize numbers very quickly. If we know that one of the prime factors of x is p, then all the prime factors of x are p plus the decomposition of x/p.

Number x cannot have more than log x prime factors, because every prime factor is >= 2. Factorization by the above method works in O(logx) time complexity. Note that consecutive factors will be presented in non-decreasing order.

# Chapter 12 Euclidean algorithm

Euclidean algorithm is one of the oldest numerical algorithms still to be in common use. It solves the problem of computing the greatest common divisor (gcd) of two positive integers.

The original version of Euclid’s algorithm is based on subtraction: we recursively subtract the smaller number from the larger.

In [84]:
def gcd(a, b):
    if a == b:
        return a
    if a > b:
        gcd(a - b, b)
    else:
        gcd(a, b - a)

Let’s estimate this algorithm’s time complexity (based on n = a+b). The number of steps can be linear, for e.g. gcd(x,1), so the time complexity is O(n). This is the worst-case complexity, because the value x + y decreases with every step

In [85]:
def gcd(a, b):
    if a % b == 0:
        return b
    else:
        return gcd(b, a % b)

This algorithm finds the gcd using only subtraction, binary representation, shifting and parity testing. We will use a divide and conquer technique. The following function calculate gcd(a, b, res) = gcd(a,b,1) · res. So to calculate gcd(a,b) it su?ces to call gcd(a, b, 1) = gcd(a,b).

In [86]:
# Greatest common divisor using binary Euclidean algorithm.
def gcd(a, b, res):
    if a == b:
        return res * a
    elif (a % 2 == 0) and (b % 2 == 0):
        return gcd(a // 2, b // 2, 2 * res)
    elif (a % 2 == 0):
        return gcd(a // 2, b, res)
    elif (b % 2 == 0):
        return gcd(a, b // 2, res)
    elif a > b:
        return gcd(a - b, b, res)
    else:
        return gcd(a, b - a, res)

This algorithm is superior to the previous one for very large integers when it cannot be assumed that all the arithmetic operations used here can be done in a constant time. Due to the binary representation, operations are performed in linear time based on the length of the binary representation, even for very big integers. On the other hand, modulo applied in algorithm 10.2 has worse time complexity. It exceeds O(logn · loglogn), where n = a + b.

# Chapter 13 Fibonacci numbers

The Fibonacci numbers form a sequence of integers defined recursively in the following way. The first two numbers in the Fibonacci sequence are 0 and 1, and each subsequent number is the sum of the previous two.

The first twelve Fibonacci numbers are:
0 1 1 2 3 5 8 13 21 34 55 89

In [ ]:
# Finding Fibonacci numbers recursively.
def fibonacci(n):
    if (n <= 1):
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

The above algorithm performs F[n] additions of 1, and, as the sequence grows exponentially, we get an ine?cient solution.
Enumeration of the Fibonacci numbers can be done faster simply by using a basis of dynamic programming. We can calculate the values F[0] ,F[1] ,...,F[n] based on the previously calculated numbers (it is sufficient to remember only the last two values).

In [88]:
# Finding Fibonacci numbers dynamically.
def fibonacciDynamic(n):
    fib = [0] *(n + 2)
    fib[1] = 1
    for i in xrange(2, n + 1):
        fib[i] = fib[i - 1] + fib[i - 2]
    return fib[n]

The time complexity of the above algorithm is O(n).

# Chapter 14 Binary search algorithm

The binary search is a simple and very useful algorithm whereby many linear algorithms can be optimized to run in logarithmic time.

Imagine the following game. The computer selects an integer value between 1 and 16 and our goal is to guess this number with a minimum number of questions. For each guessed number the computer states whether the guessed number is equal to, bigger or smaller than the number to be guessed.

1 2 3 4 5 6 7 8 9 10 **11** 12 13 14 15 16

The iterative check of all the successive values 1,2,...,16 is linear, because with each question the set of the candidates is reduced by one.
The goal is to ask a question that reduces the set of candidates maximally. The best option is to choose the middle element, as doing so causes the set of candidates to be halved each time. With this approach, we ask the logarithmic number of questions at maximum.

In [89]:
# Binary search in O(logn).
def binarySearch(A, x):
    n = len(A)
    beg = 0
    end = n - 1
    result = -1
    while (beg <= end):
        mid = (beg + end) / 2
        if (A[mid] <= x):
            beg = mid + 1
            result = mid
        else:
            end = mid - 1
    return result

The above algorithm will find the largest element which is less than or equal to x. In subsequent iterations the number of candidates is halved, so the time complexity is O(logn). It is noteworthy that the above implementation is universal; it is enough to modify only the condition inside the while loop

In many tasks, we should return some integer that is both optimal and that meets certain conditions. We can often find this number using a binary search. We guess some value and then check whether the result should be smaller or bigger. At the start we have a certain range in which we can find the result. After each attempt the range is halved, so the number of questions can be estimated by O(logn). Thus, the problem of finding the optimal value reduces to checking whether some value is valid and optimal. The latter problem is often much simpler, and the binary search adds only a logn factor to the overall time complexity.

Problem: You are given n binary values x[0] ,x[1] ,...,x[n−1] , such that x[i] ∈ {0,1}. This array represents holes in a roof (1 is a hole). You are also given k boards of the same size. The goal is to choose the optimal (minimal) size of the boards that allows all the holes to be covered by boards.

Solution: The size of the boards can be found with a binary search. If size x is sufficient to cover all the holes, then we know that sizes x+1,x+2,...,n are also sufficient. On the other hand, if we know that x is not sufficient to cover all the holes, then sizes x − 1,x − 2,...,1 are also insufficient.

In [90]:
# Binary search in O(logn).
def boards(A, k):
    n = len(A)
    beg = 1
    end = n
    result = -1
    while (beg <= end):
        mid = (beg + end) / 2
        if (check(A, mid) <= k):
            end = mid - 1
            result = mid
        else:
            beg = mid + 1
    return result

There is the question of how to check whether size x is sufficient. We can go through all the indices from the left to the right and greedily count the boards. We add a new board only if there is a hole that is not covered by the last board.

In [91]:
# Greedily check in O(n).
def check(A, k):
    n = len(A)
    boards = 0
    last = -1
    for i in xrange(n):
        if A[i] == 1 and last < i:
            boards += 1
            last = i + k - 1
    return boards

Total time complexity of such a solution is O(nlogn) due to the binary search time.

# Chapter 15 Caterpillar method

The Caterpillar method is a likeable name for a popular means of solving algorithmic tasks. The idea is to check elements in a way that’s reminiscent of movements of a caterpillar. The caterpillar crawls through the array. We remember the front and back positions of the caterpillar, and at every step either of them is moved forward.

Let’s check whether a sequence a[0] ,a[1] ,...,a[n-1] (1 <= a[i] <= 10^9) contains a contiguous subsequence whose sum of elements equals s. For example, in the following sequence we are looking for a subsequence whose total equals s = 12.

|a[0]| a[1]| a[2]| a[3]| a[4]| a[5]| a[6]| 
|---|---|---|---|---|---|---|
|6 |2 |**7** |**4** |**1** |3 |6|

Each position of the caterpillar will represent a different contiguous subsequence in which the total of the elements is not greater than s. Let’s initially set the caterpillar on the first element. Next we will perform the following steps:

* if we can, we move the right end (front) forward and increase the size of the caterpillar;
* otherwise, we move the left end (back) forward and decrease the size of the caterpillar.

In this way, for every position of the left end we know the longest caterpillar that covers elements whose total is not greater than s. If there is a subsequence whose total of elements equals s, then there certainly is a moment when the caterpillar covers all its elements.

In [93]:
def caterpillarMethod(A, s):
    n = len(A)
    front, total = 0, 0
    for back in xrange(n):
        while (front < n and total + A[front] <= s):
            total += A[front]
            front += 1
        if total == s:
            return True
        total -= A[back]
    return False

Let’s estimate the time complexity of the above algorithm. At the first glance we have two nested loops, what suggest quadratic time. However, notice that at every step we move the front or the back of the caterpillar, and their positions will never exceed n. Thus we actually get an O(n) solution. The above estimation of time complexity is based on amortized cost, which will be explained more precisely in future lessons

Problem: You are given n sticks (of lengths 1 <= a[0] <= a[1] <= ... <= a[n-1] <= 10^9). The goal is to count the number of triangles that can be constructed using these sticks. More precisely, we have to count the number of triplets at indices x < y < z, such that a[x] + a[y] > a[z]

Solution O(n^2): For every pair x,y we can find the largest stick z that can be used to construct the triangle. Every stick k, such that y < k <= z, can also be used, because the condition a[x] + a[y] > a[k] will still be true. We can add up all these triangles at once. 

If the value z is found every time from the beginning then we get a O(n^3) time complexity solution. However, we can instead use the caterpillar method. When increasing the value of y, we can increase (as far as possible) the value of z

In [95]:
# The number of triangles in O(n 2 ).
def triangles(A):
    n = len(A)
    result = 0
    for x in xrange(n):
        z = x + 2
        for y in xrange(x + 1, n):
            while (z < n and A[x] + A[y] > A[z]):
                z += 1
            result += z - y - 1
    return result

The time complexity of the above algorithm is O(n^2), because for every stick x the values of y and z increase O(n) number of times.

# Chapter 16 Greedy algorithms

Greedy programming is a method by which a solution is determined based on making the locally optimal choice at any given moment.
In other words, we choose the best decision from the viewpoint of the current stage of the solution. Depending on the problem, the greedy method of solving a task may or may not be the best approach. If it is not the best approach, then it often returns a result which is approximately correct but suboptimal. In such cases dynamic programming or brute-force can be the optimal approach. On the other hand, if it works correctly, its running time is usually faster than those of dynamic programming or brute-force.

__Coin Changing problem__

For a given set of denominations, you are asked to find the minimum number of coins with which a given amount of money can be paid. That problem can be approached by a greedy algorithm that always selects the largest denomination not exceeding the remaining amount of money to be paid. As long as the remaining amount is greater than zero, the process is repeated.

A correct algorithm should always return the minimum number of coins. It turns out that the greedy algorithm is correct for only some denomination selections, but not for all. For example, for coins of values 1, 2 and 5 the algorithm returns the optimal number of coins for each amount of money, but for coins of values 1, 3 and 4 the algorithm may return a suboptimal result. An amount of 6 will be paid with three coins: 4, 1 and 1 by using the greedy algorithm. The optimal number of coins is actually only two: 3 and 3. Consider n denominations 0 <= m[0] ‹= m[1] ‹= ... ‹= m[n−1] and the amount k to be paid.

In [96]:
# The greedy algorithm for finding change.
def greedyCoinChanging(M, k):
    n = len(M)
    result = []
    for i in xrange(n - 1, -1, -1):
        result += [(M[i], k // M[i])]
        k %= M[i]
    return result

The function returns the list of pairs: denomination, number of coins. The time complexity of the above algorithm is O(n) as the number of coins is added once for every denomination.

# Chapter 17 Dynamic programming

Dynamic programming is a method by which a solution is determined based on solving successively similar but smaller problems. This technique is used in algorithmic tasks in which the solution of a bigger problem is relatively easy to find, if we have solutions for its sub-problems.

### The Coin Changing problem

For a given set of denominations, you are asked to find the minimum number of coins with which a given amount of money can be paid. Assume that you can use as many coins of a particular denomination as necessary. The greedy algorithmic approach is always to select the largest denomination not exceeding the remaining amount of money to be paid. As long as the remaining amount is greater than zero, the process is repeated. However, this algorithm may return a suboptimal result. For instance, for an amount of 6 and coins of values 1,3,4,we get 6 = 4 + 1 + 1, but the optimal solution here is 6 = 3 + 3.

A dynamic algorithm finds solutions to this problem for all amounts not exceeding the given amount, and for increasing sets of denominations. For the example data, it would consider all the amounts from 0 to 6, and the following sets of denominations: ∅,{1},{1,3} and {1,3,4}. Let dp[i,j] be the minimum number of coins needed to pay the amount j if we use the set containing the i smallest denominations. The number of coins needed must satisfy the following rules:

* no coins are needed to pay a zero amount: dp[i,0] = 0 (for all i);
* if there are no denominations and the amount is positive, there is no solution, so for convenience the result can be infinite in this case: dp[0,j] = ∞ (for all j > 0);
* if the amount to be paid is smaller than the highest denomination c i , this denomination can be discarded: dp[i,j] = dp[i − 1,j] (for all i > 0 and all j such that c[i] > j);
* otherwise, we should consider two options and choose the one requiring fewer coins: either we use a coin of the highest denomination, and a smaller amount to be paid remains, or we don’t use coins of the highest denomination (and the denomination can thus be discarded): dp[i,j] = min(dp[i,j − c[i]] + 1,dp[i − 1,j]) (for all i > 0 and all j such that c[i] <= j).

The following table shows all the solutions to sub-problems considered for the example data.

|dp[i,j] |0 |1 |2 |3 |4 |5 |6|
|---|---|---|---|---|---|---|
|∅ |0 |∞ |∞ |∞ |∞ |∞ |∞|
|{1}| 0 |1 |2 |3 |4 |5 |6|
|{1,3}| 0| 1| 2| 1| 2| 3| 2|
|{1,3,4}| 0| 1| 2| 1| 1| 2| 2|

Consider n denominations, 0 < c[0]  <= c[1] <= ... <= c[n−1]. The algorithm processes the respective denominations and calculates the minimum number of coins needed to pay every amount from 0 to k. When considering each successive denomination, we use the previously calculated results for the smaller amounts.

In [97]:
# The dynamic algorithm for finding change.
def dynamic_coin_changing(C, k):
    n = len(C)
    # create two-dimensional array with all zeros
    dp = [[0] * (k + 1) for i in xrange(n + 1)]
    dp[0] = [0] + [MAX_INT] * k
    for i in xrange(1, n + 1):
        for j in xrange(C[i - 1]):
            dp[i][j] = dp[i - 1][j]
        for j in xrange(C[i - 1], k + 1):
            dp[i][j] = min(dp[i][j - C[i - 1]] + 1, dp[i - 1][j])
    return dp[n]

Both the time complexity and the space complexity of the above algorithm is O(n·k). In the above implementation, memory usage can be optimized. Notice that, during the calculation of dp, we only use the previous row, so we don’t need to remember all of the rows.

In [98]:
# The dynamic algorithm for finding change with optimized memory.
def dynamic_coin_changing(C, k):
    n = len(C)
    dp = [0] + [MAX_INT]*k
    for i in xrange(1, n + 1):
        for j in xrange(C[i - 1], k + 1):
            dp[j] = min(dp[j - C[i - 1]] + 1, dp[j])
    return dp

The time complexity is O(n · k) and the space complexity is O(k).

Problem: A small frog wants to get from position 0 to k (1 <= k <= 10000). The frog can jump over any one of n fixed distances s 0 ,s[1] ,..., s[n−1] (1 <= s[i] <= k). The goal is to count the number of different ways in which the frog can jump to position k. To avoid overflow, it is sufficient to return the result modulo q, where q is a given number. We assume that two patterns of jumps are different if, in one pattern, the frog visits a position which is not visited in the other pattern.

Solution O(n · k): The task can be solved by using dynamic programming. Let’s create an array dp consisting of k elements, such that dp[j] will be the number of ways in which the frog can jump to position j.

We update consecutive cells of array dp. There is exactly one way for the frog to jump to position 0, so dp[0] = 1. Next, consider some position j > 0.
The number of ways in which the frog can jump to position j with a final jump of s[i] is dp[j − s[i] ]. Thus, the number of ways in which the frog can get to position j is increased by the number of ways of getting to position j − s[i] , for every jump s[i].

More precisely, dp[j] is increased by the value of dp[j − s[i]] (for all s[i] <= j) modulo q.

In [99]:
# Solution in time complexity O(n · k) and space complexity O(k).
def frog(S, k, q):
    n = len(S)
    dp = [1] + [0]*k
    for j in xrange(1, k + 1):
        for i in xrange(n):
            if S[i] <= j:
                dp[j] = (dp[j] + dp[j - S[i]]) % q;
    return dp[k]

The time complexity is O(n·k) (all cells of array dp are visited for every jump) and the space complexity is O(k).